In [ ]:
import pandas as pd
import yfinance as yf
from datetime import datetime
from pandas import DataFrame

In [ ]:
def obter_dados_acoes(acoes):
    # Obtém a data atual (utc now)
    data_atual = datetime.utcnow().date()

    # Obtém o primeiro dia do mês atual
    primeiro_dia_mes_atual = data_atual.replace(day=1)

    # Cria um DataFrame vazio para armazenar os dados
    dados_acoes = DataFrame(columns=["Ação", "Data", "Cotação", "Cotação Início do Mês", "Variação (%)"])

    # Loop para buscar dados de cada ação
    for acao in acoes:
        try:
            # Obtém os dados da ação usando a biblioteca Yahoo Finance
            acao_info = yf.Ticker(acao)

            # Obtém a cotação do dia atual
            cotação_atual = acao_info.history(period="1d")

            # Obtém a cotação no início do mês
            cotação_inicio_mes = acao_info.history(period="1d", start=primeiro_dia_mes_atual)

            # Calcula a variação em porcentagem
            variação_percentual = ((cotação_atual["Close"].iloc[0] - cotação_inicio_mes["Close"].iloc[0]) / cotação_inicio_mes["Close"].iloc[0]) * 100

            # Cria um DataFrame temporário com os dados da ação atual
            dados_temporarios = pd.DataFrame({"Ação": [acao], "Data": [data_atual], "Cotação": [cotação_atual["Close"].iloc[0]], "Cotação Início do Mês": [cotação_inicio_mes["Close"].iloc[0]], "Variação (%)": [variação_percentual]})

            # Concatena os dados temporários ao DataFrame principal
            dados_acoes = pd.concat([dados_acoes, dados_temporarios], ignore_index=True)

        except Exception as e:
            print(f"Erro ao obter dados para a ação {acao}: {str(e)}")

    return dados_acoes

In [ ]:
# Lista de ações da B3 que você deseja acompanhar com ".SA" no final
acoes = [
    'TGAR11.SA',
    'BPFF11.SA',
    'TRXF11.SA',
    'PVBI11.SA',
    'BTLG11.SA',
    'LVBI11.SA',
    'VILG11.SA',
    'IRDM11.SA',
    'MCCI11.SA',
    'MXRF11.SA',
    'RECR11.SA',
    'MALL11.SA',
    'XPML11.SA',
    'RBVA11.SA',
    'BBDC4.SA'
]

# Chama a função para obter os dados das ações
df_dados_das_acoes = obter_dados_acoes(acoes)

In [ ]:
try:
  df_dado_acoes_passado = pd.read_parquet("/content/dados_acoes.parquet")
  df_merge =  pd.concat([df_dado_acoes_passado, df_dados_das_acoes])
  df_final = df_merge.drop_duplicates()
  df_final.to_parquet("/content/dados_acoes.parquet")
except:
  print('Primeira carga')
  df_final = df_dados_das_acoes
  df_final.to_parquet("/content/dados_acoes.parquet")


Primeira carga


In [ ]:
resultado = pd.read_parquet("/content/dados_acoes.parquet")
resultado

,Ação,Data,Cotação,Cotação Início do Mês,Variação (%)
0,TGAR11.SA,2023-09-07,122.599998,121.949997,0.533007
1,BPFF11.SA,2023-09-07,72.900002,72.830002,0.096114
2,TRXF11.SA,2023-09-07,113.500000,113.000000,0.442478
3,PVBI11.SA,2023-09-07,103.440002,103.199997,0.232563
4,BTLG11.SA,2023-09-07,104.430000,104.290001,0.134240
5,LVBI11.SA,2023-09-07,117.559998,116.231377,1.143083
6,VILG11.SA,2023-09-07,103.459999,105.800003,-2.211724
7,IRDM11.SA,2023-09-07,83.180000,82.500000,0.824243
8,MCCI11.SA,2023-09-07,95.620003,93.199997,2.596573
9,MXRF11.SA,2023-09-07,10.780000,10.760000,0.185869
